In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

In [2]:
sectors = gpd.read_parquet('industrial.parquet')
osmsites = gpd.read_parquet('is_nl33.parquet')

In [3]:
def firm_stat_calc(set_no, RP, spar, nos, sectors, osmsites, eac):
    path = os.path.join(os.getcwd(), f'Set{set_no}\\src\\results\\osm_industrial_1.01_0.25_mosek_{RP}_{spar}_{nos}.xlsx')
    osm_df = pd.read_excel(path)

    sectors = sectors.to_crs(28992)
    osmsites = osmsites.to_crs(28992)

    osmsites['area'] = osmsites.area
    osmsites['flood_fail'] = osm_df['flood_fail']
    osmsites['power_fail'] = osm_df['power_fail']
    
    sectors_centroids = sectors.copy()
    sectors_centroids['geometry'] = sectors_centroids.geometry.centroid
    
    sectors_in_osmsite = gpd.sjoin(sectors_centroids, osmsites[['geometry', 'osmid', 'flood_fail', 'power_fail', 'area']],how='inner',predicate='within')
    # Only power failed
    sectors_pf = sectors_in_osmsite[sectors_in_osmsite['power_fail'] == 0]
    number_pf  = len(sectors_pf)
    flood_and_power =sectors_in_osmsite[(sectors_in_osmsite['power_fail'] == 0) & (sectors_in_osmsite['flood_fail'] == 0)]
    number_fandp  = len(flood_and_power)
    eai = eac.loc[nos,spar]
    a = [RP ,spar, nos, set_no, number_pf, number_fandp, eai]
    return a

In [4]:
stats = []
RPS = [1000, 10000]
sparseness = [0.01, 0.2, 0.4, 0.6, 0.8]
nos_total = np.arange(0,50, 1)
total_iterations = 8 * len(RPS) * len(sparseness) * len(nos_total)

with tqdm(total=total_iterations, desc="Processing", ncols=100) as pbar:
    for i in range(8):
        set_no = i+1
        eac_path = os.path.join(os.getcwd(), f'Set{set_no}\\src\\eac_df_sparseness.xlsx')
        eac = pd.read_excel(eac_path, index_col = 0)
    
        for j in range(len(RPS)):
            for k in range(len(sparseness)):
                for m in range(len(nos_total)):
    
                    RP = RPS[j]
                    spar = sparseness[k]
                    nos = nos_total[m]
    
                    a = firm_stat_calc(set_no, RP, spar, nos, sectors, osmsites, eac)
                    stats.append(a)
                    pbar.update(1)

Processing: 100%|███████████████████████████████████████████████| 4000/4000 [29:11<00:00,  2.28it/s]


In [5]:
df_stats = pd.DataFrame(stats, columns = ['RP' ,'spar', 'nos', 'set_no', 'number_pf', 'number_fandp', 'eai'])
df_stats

,RP,spar,nos,set_no,number_pf,number_fandp,eai
0,1000,0.01,0,1,143,18,11.587235
1,1000,0.01,1,1,44,0,5.123656
2,1000,0.01,2,1,88,0,7.950613
3,1000,0.01,3,1,54,12,6.096158
4,1000,0.01,4,1,58,22,8.104171
...,...,...,...,...,...,...,...
3995,10000,0.80,45,8,305,136,3.398608
3996,10000,0.80,46,8,279,144,3.610827
3997,10000,0.80,47,8,272,141,4.987721
3998,10000,0.80,48,8,253,117,3.364988


In [6]:
df_stats.to_parquet('df_stats.parquet')